# ロボットマニピュレータの運動学

吉田勝俊（宇都宮大学）

## 参考情報
- [ipywidgetsでインタラクティブなグラフを作る - Qiita](https://qiita.com/studio_haneya/items/adbaa01b637e7e699e75)
- [Widget List &mdash; Jupyter Widgets 8.0.0a5 documentation](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)

In [ ]:
import numpy as np                        #数値計算ライブラリ
from math import sin, cos, pi             #低機能だが計算が速い数学関数
import matplotlib.pyplot as plt           #描画ライブラリ
from mpl_toolkits.mplot3d import Axes3D   #3次元座標用のモジュール
import ipywidgets as ipw                  #対話的処理のモジュール
#↓Colab用の設定（グラフィックのインライン表示）
%matplotlib inline

## 同次変換

### 同次変換を操作するユーザ関数

In [ ]:
def H_3d(angle, axis, r):
  '''
  同次変換行列(3次元)を作る
  '''
  a1, a2, a3 = axis #回転軸の成分
  aa1 = a1*a1 #2乗を作っておく
  aa2 = a2*a2
  aa3 = a3*a3
  C = cos(angle)
  S = sin(angle)

  H = np.array([
    [ aa1 + (1-aa1)*C,    a1*a2*(1-C) - a3*S, a1*a3*(1-C) + a2*S, r[0] ],              
    [ a1*a2*(1-C) + a3*S, aa2 + (1-aa2)*C,    a2*a3*(1-C) - a1*S, r[1] ],              
    [ a1*a3*(1-C) - a2*S, a2*a3*(1-C) + a1*S, aa3 + (1-aa3)*C,    r[2] ],              
    [                  0,                  0,               0,      1  ],              
  ])

  return H

def add_dummy(x):
  '''
  物理座標にダミー成分を追加する関数
  '''
  # return np.array([x[0], x[1], x[2], 1]) #ダミー成分を追加
  return np.append(x, 1) #同じ処理のPython的な書き方

def remove_dummy(xx):
  '''
  同次座標からダミー変数を除去する関数
  '''
  # return np.array([xx[0], xx[1], xx[2]]) #ダミー成分を除去
  return xx[:-1] #同じ処理のPython的な書き方

お試し

In [ ]:
H = H_3d(pi/2, [1,0,0], [1,2,3])
print( '(同次変換行列)\n', H )

x = np.array([1,2,3])
print( '(物理座標)\n', x )

hx = add_dummy(x)
print( '(物理座標→同次座標)\n', hx )

hy = np.dot(H, hx) #同次変換行列と同次座標の積
print( '(同次変換行列×同次座標)\n', hy )

y = remove_dummy(hy)
print( '(同次座標→物理座標)\n', y )

- 【注意】 `6.123234e-17` は $6.123234\times10^{-17}$ のコンピュータ表記．計算機誤差を含む 0 のこと．

### 表 6.1 の同次変換行列

In [ ]:
def Trans(r):
  '''
  平行移動
  '''
  return H_3d(angle=0, axis=[1,0,0], r=r) #axis の方向は任意

def Rotx(angle):
  '''
  X軸回転
  '''
  return H_3d(angle=angle, axis=[1,0,0], r=np.zeros(3))

def Roty(angle):
  '''
  Y軸回転
  '''
  return H_3d(angle=angle, axis=[0,1,0], r=np.zeros(3))

def Rotz(angle):
  '''
  Z軸回転
  '''
  return H_3d(angle=angle, axis=[0,0,1], r=np.zeros(3))

お試し

In [ ]:
print( '(平行移動)\n', Trans([1,2,3]) )
print( '(X軸回転)\n', Rotx(pi/3) )
print( '(Y軸回転)\n', Roty(pi/3) )
print( '(Z軸回転)\n', Rotz(pi/3) )

## 応用例 ― 4 自由度マニピュレータ

### 手先の形状データ集合（あらかじめ同次座標で与える場合）

In [ ]:
hand = np.array([
  [0,    0,    0,  1],  #[x,y,z,1]　1点目
  [0, -0.2,  0.2,  1],  #2点目
  [0,  0.2,  0.2,  1],  #3点目
  [0,    0,    0,  1]   #4点目 = 1点目 （一筆書きの都合）
])

### マニピュレータの形状データ集合

In [ ]:
def get_manipulator_points( hand, angles ):
  '''
  手先の形状データhandと，各関節角anglesから，
  マニピュレータの形状データ集合を求める
  '''
  l1, l2, l3 = 0.8, 0.8, 0.8   #リンク長
  th1, th2, th3, th4 = angles  #関節角
  origin = np.array([0,0,0,1]) #原点の同時座標

  # 使用する同次変換行列
  R4 = Rotz( th4 )
  T3 = Trans([0,0,l3])
  R3 = Rotx( -th3 )
  T2 = Trans([0,0,l2])
  R2 = Rotx( -th2 )
  T1 = Trans([0,0,l1])
  R1 = Rotz( th1 )

  # 式(7.11)の形状データの作成

  ##手先　※行列の積np.dot(A,B)は，A.dot(B)という文法でも書けます 
  H = R1.dot(T1).dot(R2).dot(T2).dot(R3).dot(T3).dot(R4)
  xi1s = [] #空のリスト
  for point in hand:  #手先
    newpoint = np.dot(H, point)
    xi1s.append(newpoint) #計算結果をリストに追加
  xi1s = np.array(xi1s) #リストを配列化

  ##肘
  H = R1.dot(T1).dot(R2).dot(T2)
  xi4 = np.dot(H, origin)

  ##肩
  H = R1.dot(T1)
  xi5 = np.dot(H, origin)

  ##根本
  xi6 = origin

  ##一筆書きの点列
  allpoints = np.vstack((xi1s, xi4, xi5, xi6))

  ##ダミー成分(最終列)の除去
  allpoints = allpoints[:,:-1]

  return allpoints

get_manipulator_points(hand, [pi/2,pi/2,pi/2,pi/2]) #お試し

## 手動操作

【注意】簡易的な実装のため，滑らかには動きません．

In [ ]:
def draw_body(th1deg, th2deg, th3deg, th4deg):
  '''
  関節角（４つ）の姿勢でマニピュレータを描く関数
  '''
  fig = plt.figure(figsize=(5,5))             #正方形のキャンバスに，
  ax = fig.add_subplot(111, projection='3d')  #3次元グラフ用紙を作る

  ax.set_title('angles = (%d, %d, %d, %d)'%(th1deg, th2deg, th3deg, th4deg))
  angles_rad = np.array([th1deg, th2deg, th3deg, th4deg])/180*pi

  newpoints = get_manipulator_points( hand, angles=angles_rad )
  xs = newpoints[:,0] #1列目　x座標
  ys = newpoints[:,1] #2列目　y座標
  zs = newpoints[:,2] #3列目　z座標

  ax.plot(xs, ys, zs)
  ax.set_xlim([-0.5, 2])            #x軸の範囲
  ax.set_ylim([-0.5, 2])            #y軸の範囲
  ax.set_zlim([   0, 2.5])          #z軸の範囲
  ax.set_xlabel('X')                #x軸のラベル
  ax.set_ylabel('Y')                #y軸のラベル
  ax.set_zlabel('Z')                #x軸のラベル
  ax.grid()

  fig.tight_layout()
  # plt.show()

### UI(ユーザーインターフェイス)を作る
#スライダの定義
slider_th1 = ipw.IntSlider(description='th1', 
                min=-180, max=180, step=10, value=-40) #下限,上限,刻み,初期値
slider_th2 = ipw.IntSlider(description='th2',
                min=-180, max=180, step=10, value=60)
slider_th3 = ipw.IntSlider(description='th3',
                min=-180, max=180, step=10, value=30)
slider_th4 = ipw.IntSlider(description='th4',
                min=-180, max=180, step=10, value=90)

#スライダを縦に並べるUI
ui = ipw.VBox([slider_th1, slider_th2, slider_th3, slider_th4])

#スライダの値でグラフを更新するオブジェクト
out = ipw.interactive_output( 
    draw_body, 
    { 
      'th1deg': slider_th1, 
      'th2deg': slider_th2, 
      'th3deg': slider_th3,
      'th4deg': slider_th4,
    }
)

out.layout.height = '350px' #サイズ固定で若干チラツキが緩和
out.layout.width  = '350px'
display(ui, out)            #UIとグラフを出力